Import important library for creating dashboard


In [1]:
import numpy
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash import Input, Output
from dash import dash_table
import pandas as pd
import dash_bootstrap_components as dbc


Read record of travel history and show example of the dataset

In [2]:
df = pd.read_csv('travel_history.csv')
df

,agentID,occupation,date,start,destination,health_stage
0,4,Officer,1,0,4,Healthy
1,6,Officer,1,0,5,Healthy
2,7,Officer,1,0,2,Healthy
3,9,Officer,1,0,4,Healthy
4,13,Officer,1,0,2,Healthy
...,...,...,...,...,...,...
150005,2475,Unemployed,365,1,7,Healthy
150006,2478,Unemployed,365,7,2,Healthy
150007,2481,Unemployed,365,4,7,Healthy
150008,2486,Unemployed,365,2,1,Healthy


group the dataset by date and Travel Route(Start-Destination) and count each health stage of traveller

this process is performed to get the summary of the mobility in each date

In [3]:
route_df = df.groupby(['date','start','destination'])['health_stage'].value_counts().unstack(fill_value=0).reset_index()
date = route_df["date"]
start = route_df["start"]
destination = route_df["destination"]
Healthy =  route_df["Healthy"]
Incubating =  route_df["Incubating"]
Infected =  route_df["Infected"]
data = {"Date": date, "Start": start, "Destination": destination, "Healthy": Healthy, "Incubating": Incubating, "Infected": Infected }
new_route_df = pd.DataFrame(data)
new_route_df

,Date,Start,Destination,Healthy,Incubating,Infected
0,1,0,1,4,0,0
1,1,0,2,23,0,0
2,1,0,3,16,0,0
3,1,0,4,4,0,0
4,1,0,5,3,0,0
...,...,...,...,...,...,...
8013,365,5,7,3,0,0
8014,365,6,7,22,0,0
8015,365,7,2,26,0,0
8016,365,7,4,27,0,0


In [4]:
new_route_df["Route"] =  new_route_df['Start'].astype(str) + '->' + new_route_df['Destination'].astype(str)
new_route_df

,Date,Start,Destination,Healthy,Incubating,Infected,Route
0,1,0,1,4,0,0,0->1
1,1,0,2,23,0,0,0->2
2,1,0,3,16,0,0,0->3
3,1,0,4,4,0,0,0->4
4,1,0,5,3,0,0,0->5
...,...,...,...,...,...,...,...
8013,365,5,7,3,0,0,5->7
8014,365,6,7,22,0,0,6->7
8015,365,7,2,26,0,0,7->2
8016,365,7,4,27,0,0,7->4


Read record of cluster state and create figure for geographical visualization

In [5]:
import plotly.express as px

cluster_detail= pd.read_csv('cluster_detail.csv')
cluster_detail["population"] = cluster_detail[["Healthy","Incubating","Infected"]].sum(axis=1)
#cluster_detail = cluster_detail[cluster_detail["Date"]==1]
fig = px.scatter_mapbox(cluster_detail[cluster_detail["Date"]==0], lat="lat", lon="lon",size="Infected", hover_name="Cluster", hover_data=["Risk", "population"],
                        color_discrete_sequence=["red"], zoom=8, height=400)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
cluster_detail = cluster_detail.rename(
    columns={'M_Suscept': 'Mosquitoes_Suscept', 'M_Latent': 'Mosquitoes_Latent', 'M_Infected': 'Mosquitoes_Infected'})

Create the function for drawing the geographical plot on dashboard

In [6]:

def drawMap():
    return  html.Div([
        dbc.Card(
            dbc.CardBody([
                html.H4("Area with Infected agent"),
                dcc.Graph(
                    id="mapviz",
                    figure=fig.update_layout(
                        template='plotly_dark',
                        plot_bgcolor= 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                    ),
                    config={
                        'displayModeBar': False
                    }
                )
            ])
        ),
    ])

Create the function for create the table plot, contain detail of each cluster, on dashboard

In [7]:

def drawDatatable():
    data_df = cluster_detail[cluster_detail.Date == 0]
    data_df = data_df.filter(["Cluster",'Healthy','Incubating','Infected',"Mosquitoes_Suscept","Mosquitoes_Latent","Mosquitoes_Infected",'Risk'])
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.H4("Daily Cluster Detail"),
                dash_table.DataTable(
                    data_df.to_dict('records'),
                    [{"name": i, "id": i} for i in data_df.columns],
                    id="data_table"

                ,style_as_list_view=True)
            ])
        ),
    ])

create line + scatter plot for risk, agent population visualization

In [8]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig_risk = px.line(cluster_detail[cluster_detail["Date"]==0], x='Date', y='Risk', color='Cluster', markers=True, title="Cluster Risk overview")
#fig_risk = fig_risk.update_xaxes(range = [0,100])

fig_pop = px.line(cluster_detail[cluster_detail["Date"]==0], x='Date', y='Mosquitoes_Suscept' , color='Cluster', markers=True, title="Population of Mosquitoes overview")
#fig_pop = fig_risk.update_xaxes(range = [0,100])

fig_h_suscept = px.line(cluster_detail[cluster_detail["Date"]==0], x='Date', y='Healthy' , color='Cluster', markers=True, title="Volumn of Susceptible Agent on each cluster")
fig_h_incubate = px.line(cluster_detail[cluster_detail["Date"]==0], x='Date', y='Incubating' , color='Cluster', markers=True, title="Volumn of Latent Agent on each cluster")
fig_h_infect = px.line(cluster_detail[cluster_detail["Date"]==0], x='Date', y='Infected' , color='Cluster', markers=True, title="Volumn of Infected Agent on each cluster")

def drawRiskmap():
    date_df = cluster_detail[cluster_detail.Date == 0]
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                dcc.Graph(
                    id="riskviz",
                    figure=fig_risk.update_layout(
                        template='plotly',
                    ),
                    config={
                        'displayModeBar': False
                    }
                )
            ])
        ),
    ])


def drawPopmap():
    date_df = cluster_detail[cluster_detail.Date == 0]
    return html.Div([

        dbc.Card(
            dbc.CardBody([
                dcc.Graph(
                    id="popviz",
                    figure=fig_pop.update_layout(
                        template='plotly',
                    ),
                    config={
                        'displayModeBar': False
                    }
                )
            ])
        ),
    ])

def draw_H_sus():
    date_df = cluster_detail[cluster_detail.Date == 0]
    return html.Div([

        dbc.Card(
            dbc.CardBody([
                dcc.Graph(
                    id="h_suscept_viz",
                    figure=fig_h_suscept.update_layout(
                        template='plotly',
                    ),
                    config={
                        'displayModeBar': False
                    }
                )
            ])
        ),
    ])
def draw_H_incubate():
    date_df = cluster_detail[cluster_detail.Date == 0]
    return html.Div([

        dbc.Card(
            dbc.CardBody([
                dcc.Graph(
                    id="h_incubate_viz",
                    figure=fig_h_incubate.update_layout(
                        template='plotly',
                    ),
                    config={
                        'displayModeBar': False
                    }
                )
            ])
        ),
    ])
def draw_H_infect():
    date_df = cluster_detail[cluster_detail.Date == 0]
    return html.Div([

        dbc.Card(
            dbc.CardBody([
                dcc.Graph(
                    id="h_infect_viz",
                    figure=fig_h_infect.update_layout(
                        template='plotly',
                    ),
                    config={
                        'displayModeBar': False
                    }
                )
            ])
        ),
    ])

Create table for visualize the travel history of the agent

In [9]:
table_header = [
    html.Thead(html.Tr([html.Th("Route"), html.Th("Healthy"),html.Th("Incubating"),html.Th("Infected")]))
]
color_map = ["red","green","orange","yellow","pink","blue","purple","navy"]
row = []

for i, record in new_route_df[new_route_df.Date == 1].iterrows() :
    row.append(html.Tr([html.Td([dbc.Badge(str(record['Start']),pill=True,color=color_map[record["Start"]] ),html.P(" -> "), dbc.Badge(str(record['Destination']),pill=True,color=color_map[record["Destination"]] )]), html.Td(record['Healthy']), html.Td(record['Incubating']), html.Td(record['Infected'])]))

table_body = [html.Tbody(row)]

def drawroutetable():
    data_df = new_route_df[new_route_df.Date == 1]
    data_df = data_df.filter(["Route","Healthy","Incubating","Infected"])
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.H4("Daily Transmission"),
                #dbc.Table(table_header + table_body, bordered=True)
                dash_table.DataTable(
                    data_df.to_dict('records'),
                    [{"name": i, "id": i} for i in data_df.columns],
                    # columns=[
                    #     {"id": "Start", "name": "Start", "presentation": "markdown"},
                    #     {"id": "Route", "name": "Route", "presentation": "markdown"},
                    #     {"id": "Destination", "name": "Destination", "presentation": "markdown"},
                    #     {"id": "Healthy", "name": "Healthy"},
                    #     {"id": "Incubating", "name": "Incubating"},
                    #     {"id": "Infected", "name": "Infected"},
                    # ],markdown_options={"html": True},
                    id="route_table"
                    ,style_as_list_view=True)

            ])
        ,style={"height": "570px","overflow-x": "scroll"}),
    ])

Initiate the dashboard application & insert visualization to card element in dashboard

In [10]:
import math
def cal_accum_risk(series):
    inverse_risk_series = [1-x for x in series]
    risk_prod = math.prod(inverse_risk_series)
    if(risk_prod)> 1:
        print(inverse_risk_series)
        print(risk_prod)
    return 1- risk_prod

In [11]:
import plotly.express as px

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Create server variable with Flask server object for use with gcorn
server = app.server
df = px.data.iris()
def drawText():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H2("Malaria Transmission on Human Mobility"),
                    #html.H4("Risk Calculation")
                ], style={'textAlign': 'center'})
            ])
        ),
    ])

def drawFigure():
    return  html.Div([
        dbc.Card(
            dbc.CardBody([
                dcc.Graph(
                    figure=px.bar(
                        df, x="sepal_width", y="sepal_length", color="species"
                    ).update_layout(
                        template='plotly_dark',
                        plot_bgcolor= 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                    ),
                    config={
                        'displayModeBar': False
                    }
                )
            ])
        ),
    ])


app.layout = html.Div([

    # dbc.NavbarSimple(dbc.Container(
    #     [
    #         dbc.Row([
    #             dbc.Col([
    #                 html.H1("Date of Simulation")
    #
    #             ], width=2),
    #             dbc.Col([
    #                 html.Div(dcc.Slider(  #slider
    #                     id='date-slider',
    #                     min=cluster_detail["Date"].min(),
    #                     max=cluster_detail["Date"].max(),
    #                     value=0,
    #                     tooltip={"placement": "bottom", "always_visible": True},
    #                     step=None
    #                 ), style={'width': '75%', 'padding': '20px 20px 20px 20px'})
    #
    #             ], width=10)
    #             ],
    #             align="center",
    #         ),
    #
    #      ]
    # ), fixed="bottom"),
    dbc.Card(
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    drawText(),

                ], width=12),
            ], align='center'),
            html.Br(),
            dbc.Row([
                dbc.Col([
                    drawMap()
                ], width=7),
                dbc.Col([
                    drawroutetable(),

                ], width=5)
            ], align='center'),
            html.Br(),
            dbc.Row([
                dbc.Col([
                    drawRiskmap()
                ], width=6),
                dbc.Col([
                    drawPopmap()
                ], width=6),
            ], align='center'),
            html.Br(),
            dbc.Row([
                dbc.Col([
                   draw_H_sus()
                ], width=4),
                dbc.Col([
                    draw_H_incubate()
                ], width=4),
                dbc.Col([
                    draw_H_infect()
                ], width=4),
            ], align='center'),
            html.Br(),
            dbc.Row([
                dbc.Col([
                    drawDatatable()
                ], width=12),
            ]),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
        ]), color = '#eff7fa'
    ),
    dbc.Toast( dbc.Container([html.Div(dcc.RangeSlider(  #slider
                            id='date-slider',
                            min=cluster_detail["Date"].min(),
                            max=cluster_detail["Date"].max(),
                            value=[0,5],
                            tooltip={"placement": "bottom", "always_visible": True},
                            step=None
                        ), style={'width': '75%', 'padding': '20px 20px 20px 20px'})]),
               header="Simulation Date",
               dismissable=False,

               # top: 66 positions the toast below the navbar
               style={"position": "fixed", "bottom": "0", "width": "70%", "left": "15%"},
               )
])
@app.callback(
    [Output('mapviz', 'figure'),Output('data_table', 'data'),Output('riskviz','figure'),Output("route_table","data"),Output("popviz","figure"),Output("h_suscept_viz","figure"),Output("h_incubate_viz","figure"),Output("h_infect_viz","figure")],
    Input('date-slider', 'value'))
def update_figure(value):
    filtered_cluster_detail = cluster_detail[(cluster_detail.Date >= value[0]) & (cluster_detail.Date <= value[1])]
    acc_cluster_detail = cluster_detail[(cluster_detail.Date >= value[0]) & (cluster_detail.Date <= value[1])]
    fig = px.scatter_mapbox(filtered_cluster_detail, lat="lat", lon="lon",size="Infected", hover_name="Cluster", hover_data=["Risk", "population"],
                            color_discrete_sequence=["red"], zoom=8, height=500)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    #---------------------------------------------
    filtered_cluster_detail = cluster_detail[(cluster_detail.Date >= value[0]) & (cluster_detail.Date <= value[1])]
    filtered_cluster_detail = filtered_cluster_detail.groupby(['Cluster'],as_index=False).agg({'Healthy': 'mean','Incubating':'mean',"Infected":'mean',"Mosquitoes_Suscept":"mean","Mosquitoes_Latent":"mean","Mosquitoes_Infected": "mean","Risk": cal_accum_risk})
    #----------------------------------------------
    risk_fig = px.line(acc_cluster_detail, x='Date', y='Risk', color='Cluster', markers=True ,title= "Cluster Risk overview")

    #----------------------------------------------
    #filter_route_detail = new_route_df[new_route_df.Date == value]
    filter_route_detail = new_route_df[(new_route_df.Date >= value[0]) & (new_route_df.Date <= value[1])]
    filter_route_detail = filter_route_detail.groupby(['Route'],as_index=False).agg({'Healthy': 'sum','Incubating':'sum',"Infected":'sum'})

    #----------------------------------------------
    pop_fig = px.line(acc_cluster_detail, x='Date', y='Mosquitoes_Suscept', color='Cluster', markers=True, title="Population of Mosquitoes overview")

    #----------------------------------------------
    h_sus_fig = px.line(acc_cluster_detail, x='Date', y='Healthy', color='Cluster', markers=True, title= "Susceptible Agent on each cluster")

    #----------------------------------------------
    h_inc_fig = px.line(acc_cluster_detail, x='Date', y='Incubating', color='Cluster', markers=True,title= "Latent Agent on each cluster" )


    #----------------------------------------------
    h_inf_fig = px.line(acc_cluster_detail, x='Date', y='Infected', color='Cluster', markers=True, title= "Infected Agent on each cluster")

    return fig,filtered_cluster_detail.to_dict('records'), risk_fig, filter_route_detail.to_dict('records'), pop_fig, h_sus_fig ,h_inc_fig ,h_inf_fig

# @app.callback(
#     Output('data_table', 'data'),
#     Input('date-slider', 'value'))
# def update_table(value):
#     filtered_cluster_detail = cluster_detail[cluster_detail.Date == value]
#     return filtered_cluster_detail.to_dict('records')




In [12]:
app.run_server()

Dash app running on http://127.0.0.1:8050/


TypeError: cal_accum_risk() missing 1 required positional argument: 'series'

In [25]:
cluster_detail[cluster_detail.Date == 335 ]

,Date,Cluster,Healthy,Incubating,Infected,Mosquitoes_Suscept,Mosquitoes_Latent,Mosquitoes_Infected,Risk,lat,lon,population
2680,335,0,91,232,204,690558.840498,63030.412130,2528.039906,0.121911,17.338760,98.106891,527
2681,335,1,19,118,96,691467.070953,62137.380458,2519.872940,0.276791,17.113713,98.400515,233
2682,335,2,117,234,230,691467.662599,62195.986111,2426.794139,0.106545,17.472902,98.026301,581
2683,335,3,17,91,85,691494.755152,62282.635166,2260.065376,0.297268,17.227205,98.225675,193
2684,335,4,14,113,86,691105.385991,62353.264158,2723.930066,0.327162,17.212100,97.768400,213
2685,335,5,0,33,17,691240.064702,62295.646182,2595.303622,1.322573,16.711983,98.566046,50
2686,335,6,10,75,58,686131.505600,67189.377959,2929.826955,0.526332,17.338300,97.667000,143
2687,335,7,60,267,233,687447.797632,66041.604628,2678.081982,0.122135,17.560811,97.920610,560
